In [1]:
import pandas as pd 
import numpy as np 
import sklearn as sc
import gc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.sparse import csr_matrix # Import the converter

# import tensorflow as tf
# import keras
import matplotlib.pyplot as plt

sample = pd.read_csv("a.csv")
samps = sample.drop(['GRANITE', 'STAINLESS','GYM','DOORMAN','FURNISHED','LAUNDRY', 'CLUBHOUSE','LATITUDE','LONGITUDE','DESCRIPTION', 'GARAGE_COUNT','ADDRESS', 'COMPANY','ID','NEIGHBORHOOD','SCRAPED_TIMESTAMP','YEAR_BUILT','AVAILABLE_AT','AVAILABILITY_STATUS','ID'], axis=1)
samps['YEAR'] =pd.to_datetime(samps['DATE_POSTED'])
samps['MONTH'] = pd.to_numeric(samps['YEAR'].dt.month)
samps['YEAR'] = pd.to_numeric(samps['YEAR'].dt.year)-2014

#samps['MONTH'] = pd.to_numeric(pd.to_datetime(samps['DATE_POSTED'],dayfirst=True, format = "%m"))
samps['TIME'] = 12*samps['YEAR']-1+samps['MONTH']
one = pd.get_dummies(samps,columns = ['BUILDING_TYPE'], drop_first=False)
new_samps = pd.get_dummies(one,columns = ['GARAGE','POOL'], drop_first=True)
zen = len(samps['ZIP'].unique())
new_samps.drop(columns = [ 'BUILDING_TYPE_TIME', 'BUILDING_TYPE_MH', 'BUILDING_TYPE_TH','DATE_POSTED','YEAR','MONTH'],axis =1,inplace=True)
new_samps["logrent"] = np.log(new_samps["RENT_PRICE"])
cleaned = new_samps.groupby('ZIP').filter(lambda x: len(x) > 25)
listofzips = cleaned['ZIP'].unique()
mse_list = []
neural_mse_list = []
'''
for each in listofzips:
    cleaned1 = cleaned[cleaned['ZIP'] == each]
    X = cleaned1[['BEDS','BATHS','SQFT', 'BUILDING_TYPE_APT','BUILDING_TYPE_COMM', 'BUILDING_TYPE_CON','BUILDING_TYPE_SFR', 'GARAGE_Y', 'POOL_Y','TIME']].fillna(0)
    X_train, X_test, y_train, y_test = train_test_split(X, cleaned1['logrent'], test_size=0.2)
    y_test = np.array(y_test)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = 0
    for i in range(len(y_test)):
        mse+= (y_test[i]-y_pred[i])**2
    mse = mse/len(y_test)
    mse_list.append(mse)
    #Neural Network
    x = sc.preprocessing.StandardScaler().fit_transform(X)
    y = (np.array(cleaned1['logrent']) - np.mean(np.array(cleaned1['logrent'])))/np.std(np.array(cleaned1['logrent']))
    nX_train, nX_test, ny_train, ny_test = sc.model_selection.train_test_split(x, y, test_size=0.2)
    hidden_units = 20
    activation = 'sigmoid'
    learning_rate = 0.05
    epochs = 25
    # [5,10,25,50,75,100]
    batch_size = 32
    neural_model = keras.models.Sequential()
    neural_model.add(keras.layers.Dense(input_dim=len(X.columns),
                                 units=hidden_units,
                                 activation=activation))
    # add the output layer
    neural_model.add(keras.layers.Dense(input_dim=hidden_units,
                                 units=1))
    # define our loss function and optimizer
    neural_model.compile(loss='MeanSquaredError',
                  # Adam is a kind of gradient descent
                  optimizer=keras.optimizers.Adam(learning_rate=.01),
                  metrics=['mse'])
    execute = neural_model.fit(nX_train, ny_train, epochs=epochs, batch_size=batch_size)
    test_acc = neural_model.evaluate(nX_test, ny_test)
    neural_mse_list.append(test_acc)
average = np.mean(mse_list)
average2 = np.mean(neural_mse_list)
print(average)
print(average2)
'''
del sample, samps, one, new_samps
gc.collect()
a = pd.get_dummies(cleaned['ZIP'],drop_first = False, sparse=True)
ab = a.sparse.to_coo().tocsr()
pca = sc.decomposition.TruncatedSVD(200)
pca.fit(ab)
print(pca.explained_variance_ratio_)





#filter by number of entries
#log it 
#0-1 columns for neural network 
#MCA 
#Straight eregression 
#Bayesian Hierarchical

[0.00287017 0.00252539 0.00206288 0.00193871 0.00185328 0.00180844
 0.00175693 0.001703   0.00157267 0.00156599 0.00156418 0.00152901
 0.0014896  0.0014629  0.00146111 0.00141319 0.00139621 0.00139192
 0.00136463 0.00131672 0.0013107  0.00128942 0.00127847 0.00125118
 0.00123723 0.00122567 0.00121658 0.00119651 0.00117105 0.00112661
 0.00112608 0.00110846 0.00109631 0.00105292 0.00105147 0.00104468
 0.00101408 0.00100896 0.00099935 0.00099592 0.00099408 0.00099475
 0.00098718 0.00097706 0.00097426 0.00096316 0.00096171 0.00093275
 0.00092679 0.00090536 0.00090504 0.00089897 0.00089246 0.00088835
 0.00087804 0.00087606 0.00087343 0.00086656 0.00086573 0.00086221
 0.00086039 0.00085029 0.00084846 0.00083868 0.00082191 0.00081568
 0.0008144  0.0008109  0.00080936 0.00080838 0.00080374 0.0008032
 0.00079445 0.000793   0.00079018 0.00078607 0.00078725 0.00078613
 0.00078466 0.00078305 0.00077783 0.00077631 0.00077496 0.00077137
 0.00077099 0.00076372 0.0007626  0.0007576  0.00075414 0.00075

In [3]:
print(np.cumsum(pca.explained_variance_ratio_))

[0.00287017 0.00539555 0.00745844 0.00939715 0.01125043 0.01305888
 0.01481581 0.01651881 0.01809147 0.01965746 0.02122164 0.02275065
 0.02424025 0.02570315 0.02716426 0.02857745 0.02997366 0.03136558
 0.03273021 0.03404693 0.03535763 0.03664706 0.03792553 0.03917671
 0.04041394 0.04163962 0.0428562  0.04405271 0.04522376 0.04635038
 0.04747646 0.04858492 0.04968123 0.05073415 0.05178562 0.0528303
 0.05384438 0.05485334 0.05585269 0.05684861 0.05784269 0.05883744
 0.05982461 0.06080167 0.06177593 0.06273909 0.0637008  0.06463355
 0.06556035 0.0664657  0.06737074 0.06826971 0.06916217 0.07005052
 0.07092856 0.07180463 0.07267806 0.07354462 0.07441035 0.07527256
 0.07613295 0.07698324 0.0778317  0.07867038 0.07949228 0.08030797
 0.08112236 0.08193326 0.08274262 0.083551   0.08435474 0.08515795
 0.08595239 0.08674539 0.08753557 0.08832164 0.08910889 0.08989501
 0.09067967 0.09146272 0.09224056 0.09301687 0.09379183 0.0945632
 0.09533419 0.09609792 0.09686052 0.09761812 0.09837226 0.099126

In [3]:
conda update sklearn

error: incomplete escape \U at position 28